In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
# download sonnets.txt
!gdown --id 108jAePKK4R3BVYBbYJZ32JWUwxeMg20K

In [ ]:
# define path for file with sonnets
SONNETS_FILE = './sonnets.txt'

# read the data
with open('./sonnets.txt') as f:
    data = f.read()

# convert to lower case and save as a list
corpus = data.lower().split("\n")

In [ ]:
# tokenize the text

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1


In [ ]:
# generating a list of n-gram sequences

def n_gram_seqs(corpus, tokenizer):
    
    input_sequences = []

    for line in corpus:
      token_list = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

    return input_sequences


input_sequences = n_gram_seqs(corpus, tokenizer)

In [ ]:
# saving max length

max_sequence_len = max([len(x) for x in input_sequences])

In [ ]:
# padding tokenized sequences to the same length

def pad_seqs(input_sequences, maxlen):
    
    padded_sequences = []

    max_sequence_len = max([len(x) for x in input_sequences])
    padded_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    return padded_sequences


input_sequences = pad_seqs(input_sequences, max_sequence_len)

In [ ]:
# generating features and labels
def features_and_labels(input_sequences, total_words):
    
    import tensorflow as tf
    features = input_sequences[:,:-1]
    labels = input_sequences[:,-1]
    one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

    return features, one_hot_labels

features, labels = features_and_labels(input_sequences, total_words)

In [ ]:
# creating a model

def create_model(total_words, max_sequence_len):
    
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation = 'softmax'))

    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
model = create_model(total_words, max_sequence_len)

history = model.fit(features, labels, epochs=50, verbose=1)

In [ ]:
# results of training model

acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
# predicting words

seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100

for w in range(next_words):
    
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis=-1).item()
    output_word = tokenizer.index_word[predicted]
    seed_text += " " + output_word

print(seed_text)